In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from seaborn import heatmap
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
import plotly.express as px
from matplotlib import rcParams
import seaborn as sns
from sklearn.model_selection import KFold

In [ ]:
#Read all the Excel Files which have Report Data from 2016 to 2020

df_2020=pd.read_csv('../input/world-happiness-report/2020.csv')

df_2020['Happiness Rank'] =  range(1, len(df_2020.index)+1)

df_2020 = df_2020.rename(columns = {'Country name' : 'Country', 'Ladder score' : 'Happiness Score', 
                        'Explained by: Log GDP per capita' : 'Economy (GDP per Capita)', 'Social support' : 'Family', 
                                    'Healthy life expectancy' : 'Health (Life Expectancy)',
                        'Freedom to make life choices' : 'Freedom', 
                                    'Perceptions of corruption' : 'Trust (Government Corruption)'})

df_2020['Year']=2020


df_2019 = pd.read_csv('../input/world-happiness-report/2019.csv')
df_2019['Year']=2019
df_2019 = df_2019.rename(columns = {'Overall rank':'Happiness Rank', 'Country or region' : 'Country', 'Score' : 'Happiness Score',
                                      'GDP per capita' : 'Economy (GDP per Capita)', 'Social support' : 'Family',
                                      'Healthy life expectancy' : 'Health (Life Expectancy)','Freedom to make life choices' : 'Freedom'
                                     , 'Perceptions of corruption' : 'Trust (Government Corruption)'})



df_2018 = pd.read_csv('../input/world-happiness-report/2018.csv')
df_2018['Year']=2018
df_2018 = df_2018.rename(columns = {'Overall rank':'Happiness Rank', 'Country or region' : 'Country', 'Score' : 'Happiness Score',
                                      'GDP per capita' : 'Economy (GDP per Capita)', 'Social support' : 'Family',
                                      'Healthy life expectancy' : 'Health (Life Expectancy)',
                                      'Freedom to make life choices' : 'Freedom',
                                      'Perceptions of corruption' : 'Trust (Government Corruption)'})


df_2017 = pd.read_csv('../input/world-happiness-report/2017.csv')
df_2017 = df_2017.rename(columns = {'Happiness.Rank':'Happiness Rank', 'Happiness.Score' : 'Happiness Score', 
                                      'Economy..GDP.per.Capita.' : 'Economy (GDP per Capita)', 'Health..Life.Expectancy.' : 'Health (Life Expectancy)',
                                      'Trust..Government.Corruption.' : 'Trust (Government Corruption)', 'Dystopia.Residual' : 'Dystopia Residual'})
df_2017['Year']=2017


df_2016= pd.read_csv('../input/world-happiness-report/2016.csv')
df_2016['Year']=2016


df_2015 = pd.read_csv('../input/world-happiness-report/2015.csv')
df_2015['Year']=2015


df_all = pd.concat([df_2020,df_2019,df_2018,df_2017,df_2016,df_2015])

df_all=df_all[['Country', 'Happiness Rank', 'Happiness Score', 'Economy (GDP per Capita)',
                                   'Family', 'Health (Life Expectancy)', 'Freedom','Trust (Government Corruption)',
                                   'Generosity', 'Year']]

df_all.head()

*Information of 2020*

In [ ]:
df_2020.info()

In [ ]:
df_2020.describe()

**Top and Least Happiness Regions in df_2020**

In [ ]:
plt.figure(figsize=(20,10))
df_2020.head(20).groupby('Regional indicator').agg({'Country':'count'}).sort_values(by='Country',ascending=True).plot(kind='bar',color='b',title='TOP HAPINESS')
plt.show()

plt.figure(figsize=(20,10))
df_2020.tail(20).groupby('Regional indicator').agg({'Country':'count'}).sort_values(by='Country',ascending=True).plot(kind='bar',color='r',title='LEAST HAPINESS')
plt.show()

**Variablity of Each Metric**

In [ ]:
plt.rcParams['figure.figsize'] = (16,16)
df_2020[['Happiness Score',
        'upperwhisker', 'lowerwhisker',
       'Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)',
       'Freedom', 'Generosity', 'Trust (Government Corruption)',
       'Explained by: Social support', 'Explained by: Healthy life expectancy',
       'Explained by: Freedom to make life choices',
       'Explained by: Generosity', 'Explained by: Perceptions of corruption',
       'Dystopia + residual']].hist(color="pink");

df_2020.columns

**Least 10 Happiest Countries in df_2020**

In [ ]:
df_least = df_2020[-10:].sort_values('Happiness Score', ascending = True)
px.bar(df_least, x='Happiness Score', y='Country',orientation='h',title="Least 10 happiest countries",color_discrete_sequence =['pink']*len(df_least))

**Top 10 Happiest Countries in df_2020**

In [ ]:
df_top=df_2020.head(10).sort_values('Happiness Score', ascending = True)

px.bar(df_top, x='Happiness Score', y='Country',orientation='h',title="Top 10 happiest countries",color_discrete_sequence =['pink']*len(df_top))

**Correlation Map for Features in df_all(past 5 years)**

In [ ]:
plt.figure(figsize=(10,7))
col=df_all[['Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)','Freedom', 'Trust (Government Corruption)', 'Generosity','Happiness Score']].corr()
sns.heatmap(col,annot=True,fmt='.2f',cmap="Blues");
plt.title("Correlation of Past 5 years")
plt.show()

**Correlation Map for Features in df_2020**

In [ ]:
col=df_2020[['Logged GDP per capita', 'Explained by: Social support',
       'Explained by: Healthy life expectancy',
       'Explained by: Freedom to make life choices',
       'Explained by: Generosity', 'Explained by: Perceptions of corruption',
       'Dystopia + residual','Happiness Score']].corr()
plt.figure(figsize=(10,7))
sns.heatmap(col,annot=True,cmap="Greens")
plt.title('Correlattion of 2020 Dataset')
plt.show()

**Fill Na values with Mean**

In [ ]:
df_all.fillna((df_all.mean()), inplace = True)
df_2020.fillna((df_2020.mean()), inplace = True)
sns.heatmap(df_2020.isnull(),yticklabels = False, cbar = False, cmap = 'viridis')

**Plot and Visulaize Top 10 Countries for Each Feature**

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=2,constrained_layout=True,figsize=(10,10))
sns.barplot(x='Economy (GDP per Capita)',y='Country',
                        data=df_all.nlargest(10,'Economy (GDP per Capita)'),
                        ax=axes[0,1],palette="rocket")
sns.barplot(x='Health (Life Expectancy)' ,y='Country',
                        data=df_all.nlargest(10,'Health (Life Expectancy)'),
                        ax=axes[0,0],palette="rocket")
sns.barplot(x='Happiness Score' ,y='Country',
                        data=df_all.nlargest(10,'Happiness Score'),
                        ax=axes[1,1],palette="rocket")
sns.barplot(x='Generosity' ,y='Country',
                        data=df_all.nlargest(10,'Generosity'),
                        ax=axes[1,0],palette="rocket")
sns.barplot(x='Freedom' ,y='Country',
                        data=df_all.nlargest(10,'Freedom'),
                        ax=axes[2,1],palette="rocket")
sns.barplot(x='Trust (Government Corruption)' ,y='Country',
                        data=df_all.nlargest(10,'Trust (Government Corruption)'),
                        ax=axes[2,0],palette="rocket")

**Trend of Different Features over the Years against the Happiness Score - Analysed for Few Countries**


In [ ]:
countries=['India','United States','United Kingdom','Russia','China','Canada','Germany','France','Switzerland', 'Iceland', 'Denmark', 'Norway', 'Finland',
       'Netherlands','Japan', 'South Korea','Italy']
df_i=df_all[df_all['Country'].isin(countries)]
fig=px.line(df_i,x='Year',y='Happiness Score',color='Country',title='Countries Trend with Happiness Score')
fig.show()

fig=px.line(df_i,x='Year',y='Economy (GDP per Capita)',color='Country',title='Countries Trend with Economy (GDP per Capita)')
fig.show()

fig=px.line(df_i,x='Year',y='Freedom',color='Country',title='Countries Trend with Freedom')
fig.show()

**Designing Linear Regression Model to Predict Happiness Score**

In [ ]:
def lmmodel(df):
    '''
    INPUT - DataFrame
    OUTPUT - Returns 
    r2 score for Test Dataset 
    Length of Test Datset
    r2 score for Train Dataset
    Length of Train Dataset
    '''
    #Choosing X and Y columns Y- Happiness Score which needs to be Predicted X - Features to Train Model
    y=df['Happiness Score']
    X=df[['Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)','Freedom', 'Generosity',
           'Trust (Government Corruption)']]
    
    #Splitting Test and Train Dataset
    X_train,X_test,Y_train,Y_test= train_test_split(X,y,test_size=0.2, shuffle=True, random_state=1000)
    
    #Initiating Linear Regression Model
    lin_reg = LinearRegression()
    
    #Fit the Model
    lin_reg.fit(X_train,Y_train)
    
    #Predict the Happiness Score for Test and Train Dataset
    y_test_preds = lin_reg.predict(X_test)
    y_train_preds = lin_reg.predict(X_train)
    
    #Finding Score, Mean Squared Error and Mean Absolute Error
    score = lin_reg.score(X_test, Y_test)
    mse = mean_squared_error(Y_test, y_test_preds)
    mae = mean_absolute_error(Y_test, y_test_preds)
    
    #R2 Score for Model
    r2_test = r2_score(Y_test, y_test_preds)
    r2_train = r2_score(Y_train, y_train_preds) 
    
    #Length of Test and Train Dataset
    len_ytest = len(y_test_preds)
    len_ytrain = len(y_train_preds)
     
    
    
    return r2_test, len_ytest, r2_train, len_ytrain
    #return score, mse, mae

**Predicting Happiness Score**

In [ ]:
#Predicting Happiness Score for 2020 Dataset by using lmmodel Function

r2_test, len_ytest, r2_train, len_ytrain = lmmodel(df_2020)
print("2020 The r-squared score for the Test model using only quantitative variables was {} on {} values.".format(r2_test, len_ytest))
print("2020 The r-squared score for the Train model using only quantitative variables was {} on {} values.".format(r2_train, len_ytrain))

#Predicting Happiness Score for total Dataset by using lmmodel Function

r2_test, len_ytest, r2_train,len_ytrain = lmmodel(df_all)
print("All The r-squared score for the Test model using only quantitative variables was {} on {} values.".format(r2_test,len_ytest))
print("All The r-squared score for the Train model using only quantitative variables was {} on {} values.".format(r2_train,len_ytrain))

In [ ]:
y=df_all['Happiness Score']
X=df_all[['Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)', 'Freedom', 'Generosity', 
          'Trust (Government Corruption)']] 


#Splitting Test and Train Dataset
X_train,X_test,Y_train,Y_test= train_test_split(X,y,test_size=0.2, shuffle=True, random_state=13579)

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

**Linear Regression**

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(X_train,Y_train)
y_hat = lin_reg.predict(X_test)
score=lin_reg.score(X_test, Y_test)
mse = mean_squared_error(Y_test, y_hat)
mae = mean_absolute_error(Y_test, y_hat)
r2 = r2_score(Y_test, y_hat)
r2, mse

**Desicion Tree Regression**

In [ ]:
dtr= DecisionTreeRegressor()
dtr.fit(X_train,Y_train)
y_pred = dtr.predict(X_test)
test_mse = mean_squared_error(Y_test, y_pred)
y_pred_train = dtr.predict(X_train)
train_mse = mean_squared_error(Y_train, y_pred_train)
d=dtr.score(X_test, Y_test)
d, test_mse 

**Random Forest Regression**

In [ ]:
rf = RandomForestRegressor(n_estimators = 13579)
rf.fit(X_train, Y_train)
y_hat = rf.predict(X_test)
errors = abs(y_hat - Y_test)
acc = 1 - errors
c=rf.score(X_test, Y_test)
c, np.mean(acc)

**Polynomial Regression**

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
sns.set_style('darkgrid')
scores_list = []
pRange = range(2,6)
for i in pRange :
    poly_reg = PolynomialFeatures(degree=i)
    x_poly = poly_reg.fit_transform(X_train)
    poly_regressor = LinearRegression()
    poly_regressor.fit(x_poly,Y_train)
    y_pred = poly_regressor.predict(poly_reg.fit_transform(X_test))
    scores_list.append(r2_score(Y_test,y_pred))
plt.plot(pRange,scores_list,linewidth=2)
plt.xlabel('Degree of polynomial')
plt.ylabel('r2 score with varying degrees')
plt.show()

**K-Nearest Neigbours**

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knnRange = range(1,11,1)
score_list=[]
for i in knnRange:
    regressor_knn = KNeighborsRegressor(n_neighbors=i)
    regressor_knn.fit(X_train,Y_train)
    y_pred=regressor_knn.predict(X_test)
    score_list.append(r2_score(Y_test,y_pred))
plt.plot(knnRange,score_list,linewidth=2,color='red')
plt.xticks(knnRange)
plt.xlabel('no of neighbors')
plt.ylabel('r2 knn')
plt.show()

In [ ]:
regressor_knn= KNeighborsRegressor (n_neighbors=5)
regressor_knn.fit(X_train,Y_train)
y_pred=regressor_knn.predict(X_test)
r2_sc=r2_score(Y_test,y_pred)
print(r2_sc)

**Desicion Tree with K-Fold**

In [ ]:
folds = KFold(n_splits=5, shuffle=True, random_state=13579)
score = 0
Y_train = pd.DataFrame(data=Y_train, columns = {"Happiness Score"})
X_train = pd.DataFrame(data=X_train, columns = {'Economy (GDP per Capita)', 'Family','Health (Life Expectancy)', 'Freedom', 'Generosity', 
                                                'Trust (Government Corruption)'})
for i, (x_index, y_index) in enumerate(folds.split(X_train, Y_train['Happiness Score'])):
    print('-' * 22, i, '-' * 22)
    dtr= DecisionTreeRegressor()
    dtr.fit(X_train.iloc[x_index], Y_train['Happiness Score'].iloc[x_index])
    score += dtr.score(X_train.iloc[y_index], Y_train['Happiness Score'].iloc[y_index])
    print('score ', dtr.score(X_train.iloc[y_index], Y_train['Happiness Score'].iloc[y_index]))
    
b = score / folds.n_splits    
print('Average Accuracy', score / folds.n_splits)

**Random Forest with K-Fold**

In [ ]:
folds = KFold(n_splits=5, shuffle=True, random_state=13579)
score = 0
#Y_train = pd.DataFrame(data=Y_train, columns = {"Happiness Score"})
#X_train = pd.DataFrame(data=X_train, columns = {'Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)','Freedom', 'Generosity',
#'Trust (Government Corruption)'})
for i, (x_index, y_index) in enumerate(folds.split(X_train, Y_train['Happiness Score'])):
    print('-' * 22, i, '-' * 22)
    rfr = RandomForestRegressor(n_estimators=200, n_jobs=-1)
    rfr.fit(X_train.iloc[x_index], Y_train['Happiness Score'].iloc[x_index])
    score += rfr.score(X_train.iloc[y_index], Y_train['Happiness Score'].iloc[y_index])
    print('score ', rfr.score(X_train.iloc[y_index], Y_train['Happiness Score'].iloc[y_index]))
    
a = score / folds.n_splits
print('Average Accuracy', a)

In [ ]:
labelList = ['RandomForest', 'Decision Tree']
mylist2 = [a, b]
for i in range(0,len(mylist2)):
    mylist2[i]=np.round(mylist2[i]*100,decimals=3)
print(mylist2)

**Desicion Tree vs Random Forest Regression**

In [ ]:
plt.figure(figsize=(14,8))
ax = sns.barplot(x=labelList,y=mylist2)
plt.yticks(np.arange(0, 101, step=10))
plt.title('r2 score comparison among different regression models',fontweight='bold')
for p in ax.patches:
    width, height = p.get_width(), p.get_height()
    x, y = p.get_xy() 
    ax.annotate('{:.3f}%'.format(height), (x +0.25, y + height + 0.8))
plt.show()

**Score Comparison Among Different Regression Models**

In [ ]:
labelList = ['Linear Regression','K-Neigbors Regression', 'Random Forest', 'Random Forest with K-fold','Decision Tree','Decision Tree with K-fold']
mylist = [r2, r2_sc, c, a, d, b]
for i in range(0,len(mylist)):
    mylist[i]=np.round(mylist[i]*100,decimals=3)
print(mylist)

In [ ]:
plt.figure(figsize=(14,8))
ax = sns.barplot(x=labelList,y=mylist)
plt.yticks(np.arange(0, 101, step=10))
plt.title('r2 score comparison among different regression models',fontweight='bold')
for p in ax.patches:
    width, height = p.get_width(), p.get_height()
    x, y = p.get_xy() 
    ax.annotate('{:.3f}%'.format(height), (x +0.25, y + height + 0.8))
plt.show()